In [1]:
!pip install wget
!pip install wandb
!pip install xtarfile

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=31c28e8749adef99af7f11b0e7fcf18b60edca315a5f7a0a0f0cb558b16c1aea
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 1.8 MB 4.0 MB/s 
     |████████████████████████████████| 181 kB 65.5 MB/s 
     |████████████████████████████████| 144 kB 71.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1da012e5a3316957c42961cfd84fbbae1acf14abf59699dbc587e69bb3d913d1
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
  Created wheel for xtarfile: filename=xtarfile-0.1.0-py3-none-any.whl size=3862 sha256=4ba6df037500d83bddb7bce974d5e336b9715bf2d838d7c1ac0cf244d9167cc7
  Stored in directory: /root/.cache/pip/wheels/14/ed/

In [2]:
from wandb.keras import WandbCallback
import wandb
WANDB_LOG=True

In [3]:

import os
from os.path import exists
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import numpy as np
import keras
import tensorflow as tf
import xtarfile as tarfile
import csv
from keras import Model
from keras.layers import Layer
import keras.backend as K

### Configuration

In [4]:
START_TOKEN="0"
END_TOKEN="1"

In [5]:

import os
from os.path import exists
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import numpy as np
import keras
import tensorflow as tf
import xtarfile as tarfile
import csv
class Dataset:
    
    def downloadDataSet(self):
      cwd = os.getcwd()
     
      file_exists = exists('./dakshina_dataset_v1.0.tar')
      if(file_exists==False):
        print('downloading....')
        os.system('curl -SL https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar > dakshina_dataset_v1.0.tar')
        print('download Complete')
      extract_exists = exists('./dakshina_dataset_v1.0/')   
      if(extract_exists==False): 
        print('Extracting..') 
        with tarfile.open('dakshina_dataset_v1.0.tar', 'r') as archive:
            archive.extractall()
        print('Complete')

      print('You are all set')

    def readData(self,dir,start_token="0",end_token="1"):
      # Vectorize the data.
      input_texts=list()
      target_texts=list()
      original_input_texts=list()
      original_target_texts=list()
      input_characters = set()
      target_characters = set()
      with open(dir, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
      
      for line in lines:
          ln = line.split("\t")
          if(len(ln)==3):
            target_text=ln[0].strip()  
            input_text=ln[1].strip()
            original_input_texts.append(input_text)
            original_target_texts.append(target_text)
            input_text+=end_token
            target_text = start_token + target_text + end_token
            input_texts.append(input_text)
            target_texts.append(target_text)
            for char in input_text:
              if char not in input_characters:
                input_characters.add(char)
            for char in target_text:
             if char not in target_characters:
               target_characters.add(char)
      input_characters = sorted(list(input_characters))
      target_characters = sorted(list(target_characters))
      return original_input_texts,original_target_texts,input_texts,target_texts,input_characters,target_characters


   

    #def encode_decode_characters(self,train_input, train_target, val_input=[], val_target=[],start_token="\t",end_token="\n"):
    def encode_decode_characters(self,train_input, train_target,start_token="0",end_token="1"):
        # Returns the encoding for characters to integer (as a dictionary) and decoding for integers to characters (as a list) for in
        BLANK_CHAR=' '
        # Encoding and decoding of input vocabulary
        input_char_enc = {}
        input_char_dec = []
        max_encoder_seq_length = 1
        tmplist=[]
        for i in train_input:
            tmplist.append(i)
        """for i in val_input:
            tmplist.append(i)"""
        for string in tmplist:
            max_encoder_seq_length = max(max_encoder_seq_length, len(string))
            for char in string:
                if char not in input_char_enc:
                    input_char_enc[char] = len(input_char_dec)
                    input_char_dec.append(char)
        """if BLANK_CHAR not in input_char_enc:
            input_char_enc[BLANK_CHAR] = len(input_char_dec)
            input_char_dec.append(BLANK_CHAR)
"""
        # Encoding and decoding of target vocabulary
        target_char_enc = {}
        target_char_dec = []
        #target_char_enc[start_token] = len(target_char_dec)
        #target_char_dec.append(start_token)
        max_decoder_seq_length = 1
        for string in train_target :#+ val_target:
            max_decoder_seq_length = max(max_decoder_seq_length, len(string)+2)
            for char in string:
                if char not in target_char_enc:
                    target_char_enc[char] = len(target_char_dec)
                    target_char_dec.append(char)
        #target_char_enc[end_token] = len(target_char_dec)
        #target_char_dec.append(end_token)
        """if ' ' not in target_char_enc:
            target_char_enc[BLANK_CHAR] = len(target_char_dec)
            target_char_dec.append(BLANK_CHAR)"""

        """print("Number of training samples:", len(train_input))
        print("Number of validation samples:", len(val_input))
        print("Number of unique input tokens:", len(input_char_dec))
        print("Number of unique output tokens:", len(target_char_dec))
        print("Max sequence length for inputs:", max_encoder_seq_length)
        print("Max sequence length for outputs:", max_decoder_seq_length)"""

        return input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length
    def preprocessing(self,input_char_enc, input_char_dec, target_char_enc, target_char_dec,max_encoder_seq_length,max_decoder_seq_length,input_texts,target_texts,start_token="0",end_token="1"):
        encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype="float32")
        decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length), dtype="float32")
        decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, len(target_char_dec)), dtype="float32")
        for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t] =input_char_enc[char]
            #encoder_input_data[i, t + 1 :] = input_char_enc[start_token]

            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                decoder_input_data[i, t, ] = target_char_enc[char]
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    decoder_target_data[i, t - 1, target_char_enc[char]] = 1.0
            decoder_input_data[i, t + 1 :] = target_char_enc[end_token]
            decoder_target_data[i, t:, target_char_enc[end_token]] = 1.0
        return encoder_input_data,decoder_input_data,decoder_target_data


       
   

### Sequence to sequence With Attention

In [6]:
from keras.layers import Layer
import keras.backend as K

class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
    Credits to Tensorflow.org and  https://github.com/thushv89/attention_keras/
    """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='glorot_uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='glorot_uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='glorot_uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            # (batch_size, decoder_timesteps, decoder_hid_layer_size)
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            # (batch_size, decoder_timesteps, encoder_timesteps)
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [7]:
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils.vis_utils import plot_model
class seqTOseq:
  def parameters(self,input_char_dec,target_char_dec,\
                 num_of_encoders=1,\
                 num_of_decoders=1,\
                 latent_dim=64,\
                 inp_emb_size=64,\
                 cell_type="lstm",\
                 cell_activation="tanh",\
                 optimizer="adam",\
                 dropout=0,\
                 lr=0.001):
      self.num_encoder_tokens=len(input_char_dec)
      self.num_decoder_tokens=len(target_char_dec)
      self.latent_dim=latent_dim
      self.cell_type=cell_type
      self.optimizer=optimizer
      self.lr=lr
      self.num_of_encoders=num_of_encoders
      self.num_of_decoders=num_of_decoders
      self.dropout=dropout
      self.inp_emb_size=inp_emb_size
      self.cell_activation=cell_activation
  def model(self):
      if(self.cell_type=="rnn"):
         cell=keras.layers.SimpleRNN
      if(self.cell_type=="lstm"):
        cell=keras.layers.LSTM     
      if(self.cell_type=="gru"):
         cell=keras.layers.GRU
      # Define an input sequence and process it.
      
      encoder_input = keras.layers.Input(shape=(None,), name="input_1")
      encoder_inp_emb = keras.layers.Embedding(self.num_encoder_tokens, self.inp_emb_size, name="embedding_1")(encoder_input)
      encoder =cell(self.latent_dim, return_state=True,\
                    return_sequences=True,\
                    activation=self.cell_activation,\
                    name="encoder_1",\
                    dropout=self.dropout)        
    
      encoder_outputs, *encoder_states = encoder(encoder_inp_emb)
      for i in range(1,self.num_of_encoders):
        encoder_outputs, *encoder_states = cell(self.latent_dim,\
                                       return_state=True,\
                                       return_sequences=True,\
                                       activation=self.cell_activation,\
                                       dropout=self.dropout,\
                                       name="encoder_"+str(i+1))(
                                                        encoder_outputs,initial_state=encoder_states
                                                 )
      # Set up the decoder, using `encoder_states` as initial state.
      decoder_inputs = keras.Input(shape=(None,), name="input_2")
      #input embedding 
      decoder_inp_emb = keras.layers.Embedding(self.num_decoder_tokens, self.inp_emb_size, name="embedding_2")(decoder_inputs)
      decoder=cell(self.latent_dim ,return_sequences=True,\
                   return_state=True,\
                   name="decoder_1",\
                   activation=self.cell_activation,\
                   dropout=self.dropout)
      decoder_outputs, *decode_states=decoder(decoder_inp_emb, initial_state=encoder_states)
      
      for i in range(1,self.num_of_decoders):
        decoder_outputs, *decode_states = cell(self.latent_dim,\
                                       return_state=True,\
                                       return_sequences=True,\
                                       activation=self.cell_activation,\
                                       dropout=self.dropout,\
                                       name="decoder_"+str(i+1))(
                                                        decoder_outputs,initial_state=encoder_states)
     
      attn_out, attn_scores = AttentionLayer(name='attention_1')([encoder_outputs, decoder_outputs])        # Bahdanau Attention    
      dense_concat_input = keras.layers.Concatenate(axis=-1, name='concat_layer_1')([decoder_outputs  ,attn_out])                            
      decoder_dense = keras.layers.Dense(self.num_decoder_tokens, activation="softmax")
      decoder_outputs = decoder_dense(dense_concat_input)
      # Define the model that will turn
      # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
      model = keras.Model([encoder_input, decoder_inputs], decoder_outputs)
      if(self.optimizer=="rmsprop"):
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.lr)
      else:
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.lr)
      model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
      return model
  def modelSummary(self,model):
      plot_model(model, to_file='encoder_model.png', show_shapes=True)
      f = plt.figure()
      f.set_figwidth(20)
      f.set_figheight(20)
      plt.imshow(mpimg.imread('encoder_model.png'))
      plt.axis('off')
 

  

### Read data from dataset

In [8]:
ds=Dataset()
ds.downloadDataSet()
language='te'
train_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.train.tsv'
val_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.dev.tsv'
test_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.test.tsv'


downloading....
download Complete
Extracting..
Complete
You are all set


### Preprocessing data

In [9]:
train_original_input_texts,train_original_target_texts,train_ip_txt,train_op_txt,train_ip_chr,train_op_chr=ds.readData(train_dir)
val_original_input_texts,val_original_target_texts,val_ip_txt,val_op_txt,val_ip_chr,val_op_chr=ds.readData(val_dir)
test_original_input_texts,test_original_target_texts,test_ip_txt,test_op_txt,test_ip_chr,test_op_chr=ds.readData(test_dir)

In [10]:
"""input_char_enc, input_char_dec,\
 target_char_enc,\
target_char_dec,\
max_encoder_seq_length,\
max_decoder_seq_length\
=ds.encode_decode_characters(train_ip_txt, train_op_txt, val_ip_txt, val_op_txt)"""

input_char_enc, input_char_dec,\
target_char_enc,\
target_char_dec,\
max_encoder_seq_length,\
max_decoder_seq_length\
=ds.encode_decode_characters(train_ip_txt, train_op_txt)

##### Assigning training, validation and test encoder input, decoder input, decoder output

In [11]:

train_enc_input, train_dec_input, train_dec_target\
= ds.preprocessing(input_char_enc, input_char_dec, target_char_enc, target_char_dec,max_encoder_seq_length,max_decoder_seq_length,train_ip_txt, train_op_txt)



In [12]:
val_enc_input, val_dec_input, val_dec_target = \
ds.preprocessing(input_char_enc, input_char_dec, target_char_enc, target_char_dec,\
                 max_encoder_seq_length,max_decoder_seq_length,val_ip_txt, val_op_txt)


test_enc_input, test_dec_input, test_dec_target =\
 ds.preprocessing(input_char_enc, input_char_dec, target_char_enc, target_char_dec,max_encoder_seq_length,max_decoder_seq_length\
                  ,test_ip_txt, test_op_txt)


### Call the model

In [13]:
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, model):
        self.model = model
       
    def on_epoch_end(self, epoch, logs={}):
        key_list = list(target_char_enc.keys())
        val_list = list(target_char_enc.values())
        predicted_data=self.model.predict([val_enc_input, val_dec_input ])
        targetpredicted=[]
        itr=0
        for data in predicted_data:
          wordip=test_ip_txt[itr]
          itr+=1
          wordop=""
          for i in data:
          
            index=np.argmax(i)
            position = val_list.index(index)
            wordop+=key_list[position]

          #wordop=wordop.replace('\t', '')
          #wordop=wordop.replace('\n', '')
          wordop=wordop.replace('0', '')
          wordop=wordop.replace('1', '')
          #wordop=wordop.replace(' ', '')
          targetpredicted.append(wordop)
          count=0
          countTot=0
        for i in range(len(targetpredicted)):
          countTot+=1
          if(val_original_target_texts[i]==targetpredicted[i]):
              count+=1
        print(" val_exact_accuracy: "+str (count/countTot))
        if(WANDB_LOG):
          cnt=count/countTot
          wandb.log({"val_exact_accuracy":cnt})
        

#### Train the model

In [14]:
sweep_config = {
  "name": "DL_Assignment3_Rnn",
  "method": "bayes",
  "metric": {
      "name": "val_exact_accuracy",
      "goal": "maximize",
  },
  
  "parameters": {
        "num_of_encoders":{
          "values":[1,2,3]    
        },
        "num_of_decoders":{
            "values":[1,2,3]      

        },
        "cell_type":{
          "values":['lstm']  
        },
        
        "lr":{
          "values":[0.001,0.002]  
        },
        "optimizer":{
          "values":['adam']  
        },
        "dropout":{ "values": [0.25,0.3,0.4,0.5]},
        "latent_dim":{ "values": [256,512]},
        "inp_emb_size": {"values": [64,128,256]},
        "batch_size":{"values":[32,64,128]}
        }
    }

In [25]:


# This is the main function to use to train/fine-tune the model using wandb runs
def train_wandb():
    run = wandb.init()
  
    config=wandb.config
    # Set the run name
    name="num_of_encoders("+ str(config["num_of_encoders"]) + ")_"
    name = " num_of_decoders(" + str(config["num_of_decoders"]) + ")_"
    name += " cell_type(" + str(config["cell_type"]) + ")_"
    name += "latent_dim(" + str(config["latent_dim"])+ ")_"
    name += "lr(" + str(config["lr"])+ ")_"
    name += "optimizer(" + str(config["optimizer"]) + ")_"
    name += "dropout(" + str(config["dropout"]) + ")"
    name += "inp_emb_size(" + str(config["inp_emb_size"]) + ")_"
    name+="batch_size(" + str(config["batch_size"]) + ")"


    wandb.run.name = name[:-1]
    batch_size=config["batch_size"]
    inp_emb_size=config["inp_emb_size"]
    dropout=config["dropout"]
    optimizer=config["optimizer"]

    num_of_encoders=config["num_of_encoders"]
    num_of_decoders=config["num_of_decoders"]

    lr=config["lr"]
    latent_dim=config["latent_dim"]
    cell_type=config["cell_type"]

    
    seq=seqTOseq()
    seq.parameters(input_char_dec=input_char_dec,\
                 target_char_dec=target_char_dec,\
                 num_of_encoders=num_of_encoders,\
                 num_of_decoders=num_of_decoders,
                 latent_dim=latent_dim,\
                 inp_emb_size=inp_emb_size,\
                 cell_type=cell_type,\
                 cell_activation="tanh",\
                 optimizer=optimizer,\
                 dropout=dropout,\
                 lr=lr)

    model=seq.model()
    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
    model.fit(
      [train_enc_input, train_dec_input],
      train_dec_target,
      batch_size=batch_size,
      epochs=15,
     validation_data=([val_enc_input, val_dec_input ] ,val_dec_target),
     callbacks=[earlystop,CustomCallback(model),WandbCallback()]
    )



In [26]:
    

sweep_id = wandb.sweep(sweep_config, entity="kankan-jana", project="CS6910_Assignment3")


Create sweep with ID: ukjlroe5
Sweep URL: https://wandb.ai/kankan-jana/CS6910_Assignment3/sweeps/ukjlroe5


In [27]:

wandb.agent(sweep_id, train_wandb)

wandb: Agent Starting Run: hbdy829w with config:
wandb: 	batch_size: 128
wandb: 	cell_type: lstm
wandb: 	dropout: 0.3
wandb: 	inp_emb_size: 64
wandb: 	latent_dim: 512
wandb: 	lr: 0.001
wandb: 	num_of_decoders: 2
wandb: 	num_of_encoders: 1
wandb: 	optimizer: adam


Epoch 1/15
458/458 [==============================] - 57s 113ms/step - loss: 1.0746 - accuracy: 0.7147 - val_loss: 0.7748 - val_accuracy: 0.7806 - _timestamp: 1651499856.0000 - _runtime: 64.0000
Epoch 2/15
458/458 [==============================] - 50s 109ms/step - loss: 0.6763 - accuracy: 0.8037 - val_loss: 0.4459 - val_accuracy: 0.8663 - _timestamp: 1651499906.0000 - _runtime: 114.0000
Epoch 3/15
458/458 [==============================] - 50s 110ms/step - loss: 0.3064 - accuracy: 0.9082 - val_loss: 0.1373 - val_accuracy: 0.9580 - _timestamp: 1651499956.0000 - _runtime: 164.0000
Epoch 4/15
458/458 [==============================] - 51s 112ms/step - loss: 0.1224 - accuracy: 0.9636 - val_loss: 0.0988 - val_accuracy: 0.9695 - _timestamp: 1651500008.0000 - _runtime: 216.0000
Epoch 5/15
458/458 [==============================] - 51s 111ms/step - loss: 0.0906 - accuracy: 0.9727 - val_loss: 0.0881 - val_accuracy: 0.9722 - _timestamp: 1651500059.0000 - _runtime: 267.0000
Epoch 6/15
458/458 [=

accuracy,▁▃▆▇█████████
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▅▃▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▇██████████
val_exact_accuracy,▁▁▆▇▇████████
val_loss,█▅▂▁▁▁▁▁▁▁▁▁▁
accuracy,0.99232
best_epoch,7
best_val_loss,0.08155
epoch,12
loss,0.02562


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: qlueq5yd with config:
wandb: 	batch_size: 128
wandb: 	cell_type: lstm
wandb: 	dropout: 0.3
wandb: 	inp_emb_size: 64
wandb: 	latent_dim: 1024
wandb: 	lr: 0.001
wandb: 	num_of_decoders: 2
wandb: 	num_of_encoders: 1
wandb: 	optimizer: adam


Epoch 1/15
458/458 [==============================] - 153s 323ms/step - loss: 1.1155 - accuracy: 0.7058 - val_loss: 0.8184 - val_accuracy: 0.7667 - _timestamp: 1651500655.0000 - _runtime: 160.0000
Epoch 2/15
458/458 [==============================] - 149s 325ms/step - loss: 0.7510 - accuracy: 0.7830 - val_loss: 0.5569 - val_accuracy: 0.8364 - _timestamp: 1651500804.0000 - _runtime: 309.0000
Epoch 3/15
458/458 [==============================] - 150s 327ms/step - loss: 0.4471 - accuracy: 0.8668 - val_loss: 0.2690 - val_accuracy: 0.9176 - _timestamp: 1651500954.0000 - _runtime: 459.0000
Epoch 4/15
458/458 [==============================] - 150s 327ms/step - loss: 0.2072 - accuracy: 0.9376 - val_loss: 0.1420 - val_accuracy: 0.9570 - _timestamp: 1651501104.0000 - _runtime: 609.0000
Epoch 5/15
458/458 [==============================] - 150s 327ms/step - loss: 0.1020 - accuracy: 0.9699 - val_loss: 0.1012 - val_accuracy: 0.9692 - _timestamp: 1651501253.0000 - _runtime: 758.0000
Epoch 6/15
458/

...

Epoch 1/15
458/458 [==============================] - 59s 118ms/step - loss: 1.0702 - accuracy: 0.7152 - val_loss: 0.7862 - val_accuracy: 0.7756 - _timestamp: 1651508232.0000 - _runtime: 67.0000
Epoch 2/15
458/458 [==============================] - 51s 112ms/step - loss: 0.6779 - accuracy: 0.8038 - val_loss: 0.4726 - val_accuracy: 0.8592 - _timestamp: 1651508283.0000 - _runtime: 118.0000
Epoch 3/15
458/458 [==============================] - 52s 113ms/step - loss: 0.3482 - accuracy: 0.8963 - val_loss: 0.1715 - val_accuracy: 0.9501 - _timestamp: 1651508335.0000 - _runtime: 170.0000
Epoch 4/15
458/458 [==============================] - 51s 112ms/step - loss: 0.1471 - accuracy: 0.9571 - val_loss: 0.1107 - val_accuracy: 0.9669 - _timestamp: 1651508386.0000 - _runtime: 221.0000
Epoch 5/15
458/458 [==============================] - 52s 113ms/step - loss: 0.0985 - accuracy: 0.9708 - val_loss: 0.1072 - val_accuracy: 0.9678 - _timestamp: 1651508438.0000 - _runtime: 273.0000
Epoch 6/15
458/458 [=

accuracy,▁▃▆▇▇████████
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▅▃▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▇██████████
val_exact_accuracy,▁▁▅▇▇████████
val_loss,█▅▂▁▁▁▁▁▁▁▁▁▁
accuracy,0.99261
best_epoch,7
best_val_loss,0.08626
epoch,12
loss,0.02448


wandb: Agent Starting Run: s2x7pcli with config:
wandb: 	batch_size: 128
wandb: 	cell_type: lstm
wandb: 	dropout: 0.3
wandb: 	inp_emb_size: 64
wandb: 	latent_dim: 512
wandb: 	lr: 0.001
wandb: 	num_of_decoders: 2
wandb: 	num_of_encoders: 1
wandb: 	optimizer: adam


Epoch 1/15


wandb: Ctrl + C detected. Stopping sweep.
